In [ ]:
import torch
from pykeops.torch import LazyTensor

[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.


Nous utilisons le GPU Apple Silicon (MPS) sur Mac pour accélérer les calculs, lorsque celui-ci est disponible. Cela permet d'exploiter la puissance de la puce graphique intégrée pour des traitements plus rapides, notamment lors de l'utilisation de bibliothèques comme PyTorch et KeOps.

Pour chaque thread GPU (qui s’occupe d’un point x_i) :
- chaque thread calcule f[i] pour un i donné

 - Initialiser une variable locale : 
    fi = 0

 - Pour chaque bloc (chunk) de points y_j :
    - on charge un petit sous-ensemble de y_j dans la mémoire rapide du GPU

        y_chunk = y[j : j + chunk_size]

    - Pour chaque y_j dans ce chunk :

        - Calculer la distance euclidienne au carré :

        d2 = 0

        - Pour k dans [0 .. d-1] :

        diff = x_i[k] - y_j[k]

        d2 += diff * diff

        - Calculer la valeur du noyau gaussien :

        K = exp( -d2 / (2 * sigma²) )

    - Ajouter la contribution :

    fi += K 

 - Une fois tous les chunks traités : f[i] = fi -> on jette Dij et Kij immédiatement 

### Pseudo‑code — convolution / noyau gaussien (avec torch / numpy)

But : la "convolution gaussienne" peut désigner soit un produit de convolution sur une grille (FFT ou filtre séparables), soit une somme de noyaux entre ensembles de points (estimation de densité). Ci‑dessous on donne le pseudo‑code pour l'estimation de densité par noyau (Σ_j exp(-||x_i - y_j||² / 2σ²)), en 3 variantes : naïve, vectorisée et par chunks (GPU/mémoire).

1) Naïf (conceptuel, double boucle)
```text
pour i de 0 à N-1:
    fi = 0
    pour j de 0 à M-1:
        d2 = 0
        pour k de 0 à d-1:
            diff = x[i,k] - y[j,k]
            d2 += diff * diff
        K = exp(- d2 / (2 * sigma**2))
        fi += K
    f[i] = fi
```

2) Vectorisé (torch / numpy, broadcasting)
```text
# formes : x => (N, d), y => (M, d)
# étape 1 : créer des vues pour broadcasting
x_expanded = x[:, None, :]    # (N, 1, d)
y_expanded = y[None, :, :]    # (1, M, d)

# étape 2 : distances au carré (broadcast et somme sur la dernière dim)
D2 = ((x_expanded - y_expanded) ** 2).sum(axis=-1)   # (N, M)

# étape 3 : noyau gaussien
K = exp(- D2 / (2 * sigma**2))   # (N, M)

# étape 4 : somme sur j
f = K.sum(axis=1)   # (N,)
```

Remarques :
- En torch : utilisez les mêmes opérations (x[:, None, :] etc.). Les opérations sont effectuées sur le device du tenseur (CPU ou GPU).
- Complexité temporelle O(N*M*d). Mémoire intermédiaire D2 ou K prend O(N*M) — peut exploser pour grandes tailles.

3) Par chunks / mémoire limitée (GPU-friendly)
```text
# on parcourt y par petits paquets pour éviter d'allouer (N, M)
f = zeros(N)

pour start de 0 à M-1 step chunk_size:
    end = min(start + chunk_size, M)
    y_chunk = y[start:end, :]               # (chunk, d)
    y_chunk_exp = y_chunk[None, :, :]       # (1, chunk, d)
    # broadcasting avec x[:,None,:] déjà (N,1,d)
    D2_chunk = ((x[:, None, :] - y_chunk_exp) ** 2).sum(axis=-1)  # (N, chunk)
    K_chunk = exp(- D2_chunk / (2 * sigma**2))
    f += K_chunk.sum(axis=1)

# résultat f (N,)
```

4) Optimisations / variantes
- KeOps / LazyTensor : évite d'allouer D2/K complets en utilisant du calcul paresseux et réduit la consommation mémoire.
- Filtre gaussien sur grille 2D : utiliser convolution séparable (1D puis 1D) ou FFT pour grande grille (complexité différente).
- Précision & device : choisir dtype (float32) et device (.to('cuda') ou MPS) ; convertir pour affichage : f.cpu().numpy().
- Normalisation possible : diviser par M si on veut une densité moyenne.

5) Bonnes pratiques
- Tester avec petits N/M pour valider la logique vectorisée avant de monter en taille.
- Choisir chunk_size en fonction de la RAM/GPU pour éviter OOM.
- Utiliser KeOps si N*M trop grand et si on veut gains mémoire/temps automatiques.

La ligne suivante définit le type de données (`dtype`) utilisé pour les tenseurs PyTorch dans ce notebook. Ici, `torch.float32` indique que les valeurs des tenseurs seront stockées en précision flottante sur 32 bits, ce qui est généralement suffisant pour la plupart des calculs numériques tout en étant plus rapide et moins gourmand en mémoire que la précision double (`float64`).

In [4]:
N, M, d = 3000, 3000, 2 

In [5]:
x = torch.randn(N, d, device=device, dtype=dtype)
y = torch.randn(M, d, device=device, dtype=dtype)

In [6]:
sigma = 0.3

In [7]:
x_i = LazyTensor(x[:, None, :])   # (N, 1, d)
y_j = LazyTensor(y[None, :, :])   # (1, M, d)

In [8]:
D_ij = ((x_i - y_j) ** 2).sum(-1) #-> le sum(-1) effectue la 
#somme sur le dernier indice càd sur les j 

Pour obtenir la densité locale $( f_i )$, on effectue la somme sur chaque ligne du noyau gaussien $( K_{ij} )$, c'est-à-dire sur l'indice \( j \) pour chaque point d'évaluation $( x_i )$ :

$
f_i = \sum_{j=1}^{M} K_{ij}$

Dans le code, cela correspond à :

```python
f = K_ij.sum(dim=1)
```

Chaque valeur $( f_i )$ représente la densité locale autour du point $( x_i )$, calculée à partir de la contribution de tous les points sources $( y_j )$ via le noyau gaussien.

Affichage 

Visualisation 2D 